# Data Cleaning
The purpose of this notebook is to explore a dataset and prepare it for analysis. This involves some of the following:
* Duplicate data
* Null or nan values
* Multiple datatypes for the same feature
* Inconsistent formatting
* Outliers

## Source
The sales dataset used here is a fictional set for a coffee shop found on Kaggle:
https://www.kaggle.com/datasets/ahmedmohamed2003/cafe-sales-dirty-data-for-cleaning-training?resource=download


In [1]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import datetime

In [2]:
filepath = '/Users/charlie/Documents/ML Projects/Data, Native/dirty_cafe_sales copy.csv'

In [3]:
# Reading in the csv file. Docs show missing or invalid values as 'ERROR' or 'UNKNOWN'
df = pd.read_csv(filepath, na_values=['ERROR','UNKNOWN'])
df.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2.0,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4.0,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4.0,1.0,NaN,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2.0,5.0,10.0,NaN,NaN,2023-04-27
4,TXN_3160411,Coffee,2.0,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [97]:
# A cursory look at the missing data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    10000 non-null  object 
 1   Item              9031 non-null   object 
 2   Quantity          9521 non-null   float64
 3   Price Per Unit    9467 non-null   float64
 4   Total Spent       9498 non-null   float64
 5   Payment Method    6822 non-null   object 
 6   Location          6039 non-null   object 
 7   Transaction Date  9540 non-null   object 
dtypes: float64(3), object(5)
memory usage: 625.1+ KB


In [6]:
# Make a copy of the df to prevent errors
df_wip = df.copy()

## Dealing with Missing Values 
### String Datatype 
We've already changed the cells that had error in them, now let's tackle the columns of a string datatype. It seems there are only a few unique values that can be entered for these columns. For simplicity, let's assume it not possible to determine there value and use 'unknown' for these cells.

In [7]:
# Make sure all strings are consistent case in appropriate columns and replace NaN values with unknown for columns with string data 
str_columns = ['Item','Payment Method','Location']
df_wip[str_columns] = df_wip[str_columns].apply(lambda x: x.str.lower())
df_wip[str_columns] = df_wip[str_columns].fillna('unknown')

In [8]:
# Output
df_wip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    10000 non-null  object 
 1   Item              10000 non-null  object 
 2   Quantity          9521 non-null   float64
 3   Price Per Unit    9467 non-null   float64
 4   Total Spent       9498 non-null   float64
 5   Payment Method    10000 non-null  object 
 6   Location          10000 non-null  object 
 7   Transaction Date  9540 non-null   object 
dtypes: float64(3), object(5)
memory usage: 625.1+ KB


In [9]:
df_wip.sample(10)

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
485,TXN_8818168,juice,4.0,NaN,12.0,digital wallet,unknown,2023-07-20
6764,TXN_6513163,cake,2.0,3.0,6.0,digital wallet,takeaway,2023-04-01
6997,TXN_4937411,juice,1.0,3.0,3.0,unknown,takeaway,2023-08-21
8435,TXN_6786730,coffee,1.0,2.0,2.0,credit card,unknown,2023-04-15
2564,TXN_5734395,sandwich,NaN,4.0,20.0,cash,takeaway,2023-05-17
236,TXN_8562645,salad,NaN,5.0,NaN,unknown,in-store,2023-05-18
609,TXN_6836137,cake,4.0,3.0,12.0,unknown,in-store,2023-08-30
4063,TXN_6266147,juice,3.0,3.0,9.0,credit card,in-store,2023-01-31
225,TXN_5047447,salad,1.0,5.0,5.0,digital wallet,takeaway,2023-12-27
7906,TXN_5105810,cookie,5.0,NaN,5.0,unknown,takeaway,2023-01-17


### Numeric Columns
A snapshot of the missing data shows about 1400 rows with a null value in columns 2,3 or 4. A look at the whole dataframe above shows between 400-500 null values in each column; it seems that no more than one null value is in any given row. Perhaps the null values are spread evenly across different rows for these variables.

In [10]:
miss_data = df_wip.loc[df_wip.isna().any(axis=1)]

In [11]:
miss_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1841 entries, 2 to 9998
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    1841 non-null   object 
 1   Item              1841 non-null   object 
 2   Quantity          1362 non-null   float64
 3   Price Per Unit    1308 non-null   float64
 4   Total Spent       1339 non-null   float64
 5   Payment Method    1841 non-null   object 
 6   Location          1841 non-null   object 
 7   Transaction Date  1381 non-null   object 
dtypes: float64(3), object(5)
memory usage: 129.4+ KB


In [12]:
miss_data.sample(10)

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
574,TXN_7507962,smoothie,4.0,NaN,16.0,unknown,unknown,2023-06-22
5463,TXN_6806024,juice,NaN,3.0,6.0,credit card,in-store,2023-04-14
9596,TXN_9293877,juice,NaN,3.0,15.0,digital wallet,unknown,2023-08-24
8443,TXN_2023651,sandwich,NaN,4.0,NaN,cash,in-store,2023-05-25
2437,TXN_4452589,tea,NaN,1.5,1.5,unknown,takeaway,2023-08-22
1606,TXN_5580933,smoothie,3.0,4.0,NaN,cash,takeaway,2023-10-19
2962,TXN_9702662,unknown,4.0,NaN,16.0,unknown,takeaway,2023-07-10
1779,TXN_1546695,coffee,5.0,2.0,NaN,unknown,takeaway,2023-03-06
5911,TXN_9882485,salad,5.0,5.0,25.0,cash,unknown,NaN
7515,TXN_5757196,juice,5.0,3.0,NaN,unknown,in-store,NaN


#### Low Hanging Fruit
The price of the menu items are given in the documentation. This makes it easy to fill in the missing values in the 'price per unit' column. Always read any documentation that is available for the dataset before cleaning your it, it will make life easier. There are 533 rows that benefit from this.

In [13]:
df_wip[df_wip['Price Per Unit'].isna()]

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
56,TXN_3578141,cake,5.0,NaN,15.0,unknown,takeaway,2023-06-27
65,TXN_4987129,sandwich,3.0,NaN,NaN,unknown,in-store,2023-10-20
68,TXN_8427104,salad,2.0,NaN,10.0,unknown,in-store,2023-10-27
85,TXN_8035512,tea,3.0,NaN,4.5,cash,unknown,2023-10-29
104,TXN_7447872,juice,2.0,NaN,6.0,unknown,unknown,NaN
...,...,...,...,...,...,...,...,...
9924,TXN_5981429,juice,2.0,NaN,6.0,digital wallet,unknown,2023-12-24
9926,TXN_2464706,cake,4.0,NaN,12.0,digital wallet,takeaway,2023-11-09
9961,TXN_2153100,tea,2.0,NaN,3.0,cash,unknown,2023-12-29
9996,TXN_9659401,unknown,3.0,NaN,3.0,digital wallet,unknown,2023-06-02


#### Dictionary Mapping
It is more efficient to use a vectorized solution like `.map()` when there are multiple values for prices like in a menu. The first step in doing this is to create a map of the prices to the Item column.

In [14]:
# Dictionary and mapping to a new series of the menu item prices
item_prices = {'coffee':2, 'tea':1.5, 'sandwich':4, 'salad':5, 'cake':3, 'cookie':1, 'smoothie':4, 'juice':3}
map_prices = df_wip['Item'].map(item_prices)

Next, the missing values are replaced with values based on the mapping created. Fillna() is faster at this then looping through rows.

In [15]:
# Use fillna() to fill in missing values.
df_wip['Price Per Unit'] = df_wip['Price Per Unit'].fillna(map_prices)

In [16]:
print('The number of rows with missing prices:',len(df_wip[df_wip['Price Per Unit'].isna()]))
df_wip[df_wip['Price Per Unit'].isna()].head()

The number of rows with missing prices: 54


,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
118,TXN_4633784,unknown,5.0,NaN,15.0,unknown,in-store,2023-02-06
151,TXN_4031509,unknown,4.0,NaN,16.0,credit card,takeaway,2023-01-04
289,TXN_3495950,unknown,4.0,NaN,6.0,credit card,in-store,2023-02-19
334,TXN_2523298,unknown,4.0,NaN,6.0,unknown,in-store,2023-03-25
550,TXN_4186681,unknown,4.0,NaN,6.0,digital wallet,unknown,2023-05-24


Now there are only 54 rows with missing prices. Much better. These will be handled farther down.

#### Using Formulas for Null Values
We can determine the values of the quantity, price, and total spent, if we know at least 2 of the values; the value of the unknown can be calculated from the equation given in the documentation: `total spent = quantity * price per unit` . 
Here's a look at the number of null values in the 'Total Spent' column of the data.

In [17]:
print('There are this many null values for '"'Total Spent'"':', len(df_wip[df_wip['Total Spent'].isna()]))
df_wip[df_wip['Total Spent'].isna()].sample(5)

There are this many null values for 'Total Spent': 502


,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
6044,TXN_1900906,coffee,5.0,2.0,NaN,cash,unknown,2023-01-31
8128,TXN_6105807,smoothie,3.0,4.0,NaN,credit card,takeaway,2023-01-18
8087,TXN_1228474,smoothie,4.0,4.0,NaN,unknown,in-store,2023-11-20
2176,TXN_4522012,coffee,2.0,2.0,NaN,unknown,in-store,2023-04-13
94,TXN_6289610,juice,3.0,3.0,NaN,cash,takeaway,2023-08-07


To fill out the null values in the 'Total Value' column, each row will be traversed and a function will be applied `df.apply()` to the rows with null values. This is usefull for using a custom function such as the product of two values in the same row. Use this technique with caution since it is slower on large dataframes where a vectorized approach is preferred. On this project, there are only 10,000 rows, so this algorithm is shown as an example.

In [18]:
def total_price(row):
    if pd.isna(row['Total Spent']):
        return row['Quantity']*row['Price Per Unit']
    else:
        return row['Total Spent']

def order_qty(row):
    if pd.isna(row['Quantity']):
        return row['Total Spent']/row['Price Per Unit']
    else:
        return row['Quantity']

def unit_price(row):
    if pd.isna(row['Price Per Unit']):
        return row['Total Spent']/row['Quantity']
    else:
        return row['Price Per Unit']

In [19]:
df_wip['Price Per Unit'] = df_wip.apply(unit_price, axis=1)
df_wip['Quantity'] = df_wip.apply(order_qty, axis=1)
df_wip['Total Spent'] = df_wip.apply(total_price, axis=1)

In [20]:
df_wip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    10000 non-null  object 
 1   Item              10000 non-null  object 
 2   Quantity          9977 non-null   float64
 3   Price Per Unit    9994 non-null   float64
 4   Total Spent       9977 non-null   float64
 5   Payment Method    10000 non-null  object 
 6   Location          10000 non-null  object 
 7   Transaction Date  9540 non-null   object 
dtypes: float64(3), object(5)
memory usage: 625.1+ KB


#### Leftover Null Values
A snapshot of the dataframe above shows this is coming along fantastic. There are only a handful(23) of null values left to address. The `isna()` method shows these rows have only 1 of the 3 values so it remains null since you can't calculate with a null value.

What to do next? If the unit price was known and the quantity and total are unknown, it is hard to know how much the transaction was. The average of the total spent could be used, but then what should be done for the quantity? Should the average quantity be used? Or should it be calculated based on the known price and the average amount spent? In the rows where the known variable was quantity, similar questions arise. 

Because 23 out of 10,0000 datapoints amounts to just 0.2%, deleting these data points would not skew the analysis in a significant way.

In [21]:
df_wip = df_wip.dropna()
df_wip.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9514 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    9514 non-null   object 
 1   Item              9514 non-null   object 
 2   Quantity          9514 non-null   float64
 3   Price Per Unit    9514 non-null   float64
 4   Total Spent       9514 non-null   float64
 5   Payment Method    9514 non-null   object 
 6   Location          9514 non-null   object 
 7   Transaction Date  9514 non-null   object 
dtypes: float64(3), object(5)
memory usage: 669.0+ KB


## Duplicate Data
The data must be checked for duplicated transactions to validate the accuracy of the information. The `df.duplicated()` method can check whether any rows are duplicated exactly or if only certain columns are copied.


In [23]:
# Find duplicated rows
duplicates = df_wip[df_wip.duplicated(keep=False)]
print(duplicates)

# Find duplicated transaction ID
dup_ID = df_wip[df_wip.duplicated(subset=['Transaction ID'], keep=False)]
print(dup_ID)

# Find duplicated item, total, payment, location and date
dup_select = df_wip[df_wip.duplicated(subset=['Item','Total Spent','Payment Method','Location','Transaction Date'], keep=False)]
print(dup_select.head())

Empty DataFrame
Columns: [Transaction ID, Item, Quantity, Price Per Unit, Total Spent, Payment Method, Location, Transaction Date]
Index: []
Empty DataFrame
Columns: [Transaction ID, Item, Quantity, Price Per Unit, Total Spent, Payment Method, Location, Transaction Date]
Index: []
    Transaction ID      Item  Quantity  Price Per Unit  Total Spent  \
5      TXN_2602893  smoothie       5.0             4.0         20.0   
27     TXN_5695074     juice       4.0             3.0         12.0   
68     TXN_8427104     salad       2.0             5.0         10.0   
124    TXN_7945375     juice       3.0             3.0          9.0   
126    TXN_9646452     juice       5.0             3.0         15.0   

    Payment Method  Location Transaction Date  
5      credit card   unknown       2023-03-31  
27     credit card  takeaway       2023-04-10  
68         unknown  in-store       2023-10-27  
124        unknown   unknown       2023-01-14  
126    credit card   unknown       2023-09-16  


In [26]:
dup_select.sort_values('Transaction Date')

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
1895,TXN_6843517,sandwich,5.0,4.0,20.0,credit card,in-store,2023-01-02
828,TXN_7906197,sandwich,5.0,4.0,20.0,credit card,in-store,2023-01-02
5307,TXN_8309656,sandwich,3.0,4.0,12.0,unknown,takeaway,2023-01-04
7652,TXN_3317406,sandwich,3.0,4.0,12.0,unknown,takeaway,2023-01-04
2460,TXN_3910230,sandwich,5.0,4.0,20.0,cash,takeaway,2023-01-05
...,...,...,...,...,...,...,...,...
2474,TXN_1938071,sandwich,4.0,4.0,16.0,credit card,in-store,2023-12-25
5425,TXN_8600523,tea,3.0,1.5,4.5,unknown,unknown,2023-12-29
5137,TXN_4755934,tea,2.0,1.5,3.0,cash,unknown,2023-12-29
9401,TXN_8732716,tea,3.0,1.5,4.5,unknown,unknown,2023-12-29


There are no rows that are exactly identical. Nor are there any transactions with the same ID number. When evaluated for the item, total, payment type, location and date, there are about 400 rows that match. Without more information it is not possible to ascertain if these rows are duplicates or unique orders. Are the transaction IDs created by different registers? Was the timestamp recorded along with the transaction? 

## Time Date Data
When the date was read in to pandas, it was interpreted as string values. It can also be converted to a datetime datatype for data manipulation and analysis. `.to_datetime` is the method that completes this conversion.

In [27]:
df_wip['Transaction Date'] = pd.to_datetime(df_wip['Transaction Date'])

In [28]:
df_wip.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9514 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Transaction ID    9514 non-null   object        
 1   Item              9514 non-null   object        
 2   Quantity          9514 non-null   float64       
 3   Price Per Unit    9514 non-null   float64       
 4   Total Spent       9514 non-null   float64       
 5   Payment Method    9514 non-null   object        
 6   Location          9514 non-null   object        
 7   Transaction Date  9514 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 669.0+ KB


# Data Analysis
Now that the data has been prepared, some basic analysis can be done. What are the statistics for the average number of items or the average sales total per purchase? 

The data can be grouped into subsets, like month or item sales, that can be plotted to look for any cyclical trends.

Or we can keep track of how payment is made an ensure the proper terminals are used to support traffic and protect the identities of customers.